<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px; width: 180px">
            <img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo">
        </td>
        <td style="width: 100%">
            <div style="width: 100%; text-align:right"><font style="font-size:38px"><b>Softwaredesign</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>Mechatronik</b></font></div>
        </td>
    </tr>
</table>

---

# Travelable Distance - Fahrreichweiten-Simulation
Die Flotte an Logistikfahrzeugen Ihres Arbeitgebers soll erneuert werden. Nun stellt sich die große Frage, in welchem Ausmaß Elektro- bzw. Wasserstofffahrzeuge hier sinnvoll eingesetzt werden können.

Sie sollen nun bestimmen welche Reichweite ein solches Fahrzeug haben müsste, um tatsächlich praktikabel zu sein. Da sich das Liefergebiet prinzipiell auf ganz Europa erstreckt, sind Sie nicht mehr in der Lage dieses Problem händisch zu lösen.

Sie sollen daher basierend auf einem Datensatz mit ca. 69500 europäischen Ortschaften und Städten bestimmen wie viele Stopps bei einer Lieferung eingelegt werden müssen, um das Lieferfahrzeug voll zu tanken.

---
## Beginn der Setup-Code-Blöcke

In [95]:
import pandas as pd
import math
from math import radians, sin, cos, atan2, sqrt

### Hilfsfunktion um die Distanz zw. zwei Punkten auf der Erde zu bestimmen
Implementieren Sie die Funktion `distance_on_earth(lat1, lon1, lat2, lon2)` die aus den geographischen Koordinaten zweier Punkte auf der Erde die Distanz in Kilometern berechnet.  
Nutzen Sie hierfür die [Haversine-Formel](https://en.wikipedia.org/wiki/Haversine_formula). Dies ist zwar nur eine Näherungsformel, welche die Erde als perfekte Kugel annimmt, aber das Ergebnis ist für unsere Zwecke ausreichend genau.

In [96]:
def distance_on_earth(lat1, lon1, lat2, lon2):
    """Calculates distance between two points on earth using the haversine formula."""
    
    radius = 6371  
    diff_lat = lat2 - lat1
    diff_lon = lon2 - lon1

    lat1 = lat1 * math.pi / 180
    lon1_rad = lon1 * (math.pi / 180)
    diff_lat_rad = diff_lat * (math.pi / 180)
    lat2 = lat2 * math.pi / 180
    lon2_rad = lon2 * (math.pi / 180)
    diff_lon_rad = diff_lon * (math.pi / 180)

    a1 = math.sin(diff_lat_rad / 2) ** 2
    cos1 = math.cos(lat1)
    cos2 = math.cos(lat2)
    a2 = math.sin(diff_lon_rad / 2) ** 2

    middle_step = a1 + cos1 * cos2 * a2  
    sqrt1 = math.sqrt(middle_step)
    sqrt2 = math.sqrt(1 - middle_step)

  
    angle = 2 * math.atan2(sqrt1, sqrt2)
    result = angle * radius  

    final_distance = result

    return final_distance

# Test
assert 14 < distance_on_earth(47.296464, 11.586556, 47.259659, 11.400375) < 15
print(distance_on_earth(47.296464, 11.586556, 47.259659, 11.400375))



14.629436193849216


---
### Klasse um eine Stadt zu repräsentieren
Diese Klasse soll in der Lage sein eine Stadt mit deren Namen und Koordinaten zu repärsentierne.  
Weiter müssen zwei Städte auf Gleichheit überprüfbar sein und hier `True` zurück liefern, wenn der Name und die Koordinaten der Städte ident sind.  
Außerdem muss die Klasse hashbar sein, damit sie in einem `dict` gespeichert werden kann.

Ebenso notwendig ist eine Methode `distance_to()` mit der die Distanz zu einer anderen Stadt berechnet werden kann.

In [115]:
import math

class City:
    def __init__(self, name, lat, lon):
        self.name_of_city = name
        self.latitude = lat
        self.longitude = lon

    def distance_to(self, other):
        radius = 6371  
        lat_diff = self.latitude - other.latitude
        lon_diff = self.longitude - other.longitude
        lat_radians = math.radians(lat_diff)
        lon_radians = math.radians(lon_diff)
        
        a = math.sin(lat_radians / 2.0)**2 + math.cos(math.radians(self.latitude)) * math.cos(math.radians(other.latitude)) * math.sin(lon_radians / 2.0)**2
        c = math.atan2(math.sqrt(a), math.sqrt(1 - a))

        distance = radius * c 
        return distance

    def __eq__(self, other):
        if not isinstance(other, City): 
            return False
        return self.name_of_city == other.name_of_city and self.latitude == other.latitude and self.longitude == other.longitude

    def __hash__(self):
        return hash((self.name_of_city, self.latitude, self.longitude))

    def __str__(self):
        return f"Die Stadt {self.name_of_city} ist bei {self.latitude} und {self.longitude}"
    
    def __repr__(self):
        return self.__str__()



### Einlesen der CSV-Datei
Hier wird die Datei `cities_in_europe.csv` für Sie eingelesen und die darin enthaltenen Städte in einem Pandas `DateFrame` abgespeichert.  
Verschaffen Sie sich einen überblick darüber wie die Daten aufgebaut sind.

In [98]:
df = pd.read_csv('cities_in_europe.csv', sep=';', encoding='latin1')
df.head()


,ASCII Name,Country name EN,Country Code,Population,LABEL EN,Coordinates,Latitude,Longitude
0,Istanbul,Turkey,TR,14804116,Turkey,"41.01384, 28.94966",41.01384,28.94966
1,Moscow,Russian Federation,RU,10381222,Russian Federation,"55.75222, 37.61556",55.75222,37.61556
2,London,United Kingdom,GB,8961989,United Kingdom,"51.50853, -0.12574",51.50853,-0.12574
3,Saint Petersburg,Russian Federation,RU,5351935,Russian Federation,"59.93863, 30.31413",59.93863,30.31413
4,Ankara,Turkey,TR,3517182,Turkey,"39.91987, 32.85427",39.91987,32.85427


---
### Aufbau eines Graphen
Um unsere Daten sinnvoll miteinander in Verbindung zu bringen muss eine geeignete Datenstruktur für das Problem gewählt werden.  
In diesem Fall wird das ein Graph sein, welchen wir mittels `networkx`-Modul implementieren.

In [99]:
import networkx as nx

### Knoten des Graphen hinzufügen
Erstellen Sie einen neuen Graphen und fügen Sie die ersten `amount_of_cities` Städte hinzu. Städte sollen dabei immer als `City`-Objekt repräsentiert werden.  
Hierfür können Sie die `add_node(...)`-Methode des Graphen verwenden, wie sie in der [Vorlesung vorgestellt](https://mrp123.github.io/MCI-MECH-B-3-SWD-SWD-ILV/03_Algorithmen_und_Datenstrukturen/Examples/B%C3%A4ume_und_Graphen/networkx_example.py) wurde. Für genauere Informationen können Sie auch die [Dokumentation von `networkx`](https://networkx.org/documentation/stable/reference/classes/graph.html) zu Rate ziehen.

In [125]:
graph = nx.Graph()

amount_of_cities = 4000
cities_added = 0

for i in range(amount_of_cities):
    name = df['ASCII Name'][i]
    lat = df['Latitude'][i]
    lon = df['Longitude'][i]
    city = City(name, lat, lon)
    
    graph.add_node(city)
    cities_added += 1

print(f"Anzahl der Städte: {cities_added}")
assert amount_of_cities == len(graph.nodes)




Anzahl der Städte: 4000


---
### Kanten des Graphen hinzufügen
Wir können nun über die Knoten des Graphen iterieren und daraus die Distanze zwischen allen Städten bestimmen. Wir nehmen an, dass zwei Städte miteinander verbunden werden können, wenn die Distanz zwischen ihnen kleiner als `travel_range` ist. Das soll abbilden, dass ein Fahrzeug mit einer Reichweite von `travel_range` die beiden Städte erreichen kann.
Sollten zwei Städte innerhalb unserer bestimmten Reichweite liegen, dann kann eine Kante zwischen den beiden beteiligten Knoten hinzugefügt werden.

Um dies zu implementieren, iterieren Sie über alle Knoten des Graphen (lassen sich in eine Liste umwandeln) und bestimmen Sie für jeden Knoten die Distanz zu allen anderen Knoten.

Je nach Anzahl an Knoten kann dieser Vorgang etwas Zeit in Anspruch nehmen, da für alle möglichen Kombination aus Knoten und Kanten die Distanz bestimmt werden muss. Erst dann kann entschieden werden, ob eine Kante hinzugefügt werden kann oder nicht.

Es lassen sich sinnvollere Datenstrukturen finden, um den Aufbau der Kanten zu beschleunigen. Dies ist aber nicht Teil der Aufgabe, weiters wollen wir in der Lage sein aus unserem gesamten Datensatz an Städten und deren Distanzen je zu extrahieren die innerhalb unserer Reichweite liegen.

In [117]:
city_distances_dict = {}
cities = list(graph.nodes) #Knoten des Graphen als Liste

Sie können das `itertools`-Modul verwenden um alle möglichen Kombinationen aus Knoten zu erstellen. Hierzu können Sie die Funktion `combinations(iterable, r)` verwenden.  
Ein sinnvoller Schlüssel für das `city_distances_dict` könnte ein `Tupel` aus den beiden `City`-Objekten sein.

In [ ]:
import itertools
travel_range = 150
city_distances_dict = {}
cities = list(graph.nodes)


city_distances_dict.clear() 
for city1, city2 in itertools.combinations(cities, 2):
    distance = city1.distance_to(city2)
    city_distances_dict[(city1, city2)] = distance

print(f"Es wurden Distanzen für {len(city_distances_dict)} Kombinationen berechnet")

for (city1, city2), distance in city_distances_dict.items():
    if distance <= travel_range:
        graph.add_edge(city1, city2, weight=distance)

print(f"Der Graph hat {len(graph.edges)} Kanten")


Es wurden Distanzen für 7998000 Kombinationen berechnet
Der Graph hat 401448 Kanten


Kanten können mit der Funktion `add_edge()` hinzugefügt werden. Diese Funktion erwartet zwei Knoten. Die Distanz zwischen diesen beiden Knoten kann als optionaler Parameter `weight` übergeben werden.

---
### Bestimmen der kürzesten Routen
Sie können aus dem Graphen jene Route bestimmen die am wenigsten Zwischenstopps hat. Die Anzahl der Stopps hängt stark nicht-linear von der Reichweite des Fahrzeugs ab.

Machen Sie sich hierzu mit der [Dokumentation von `networkx`](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html) vertraut, um die richtigen Funktionen zu finden.  
Überlegen Sie sich ob für diese Anwendung die Gewichtung des Graphen bei der Pfadsuche berücksichtigt werden soll, oder nicht. Implementieren Sie zum Testen beide Varianten.

Wir wollen für den ersten Test die kürzeste Route von Malaga nach Moskau bestimmen.

In [129]:
route_start = City("Malaga", 36.72016, -4.42034)
route_end = City('Moscow', 55.75222, 37.61556)

if route_start not in graph.nodes or route_end not in graph.nodes:
    raise ValueError("Start- oder Zielstadt ist nicht im Graph enthalten. Bitte überprüfen!")

path_unweighted = nx.shortest_path(graph, source=route_start, target=route_end)
path_unweighted_length = len(path_unweighted) - 1  

path_weighted = nx.shortest_path(graph, source=route_start, target=route_end, weight='weight')
path_weighted_length = sum(
    graph.edges[path_weighted[i], path_weighted[i + 1]]['weight']
    for i in range(len(path_weighted) - 1)
)


print("Ungewichteter Pfad:", path_unweighted)
print(f"Es werden {len(path_unweighted)} Stationen (inkl. Start & Ende) angefahren und {path_unweighted_length} Zwischenstopps eingelegt.")

print("Gewichteter Pfad:", path_weighted)
print(f"Es werden {len(path_weighted)} Stationen (inkl. Start & Ende) angefahren und eine Distanz von {path_weighted_length:.2f} km zurückgelegt.")


Ungewichteter Pfad: [Die Stadt Malaga ist bei 36.72016 und -4.42034, Die Stadt Lorca ist bei 37.67119 und -1.7017, Die Stadt Castello de la Plana ist bei 39.98567 und -0.04935, Die Stadt Vic ist bei 41.93012 und 2.25486, Die Stadt Salon-de-Provence ist bei 43.64229 und 5.09478, Die Stadt Aosta ist bei 45.73764 und 7.31722, Die Stadt Ravensburg ist bei 47.78198 und 9.61062, Die Stadt Passau ist bei 48.5665 und 13.43122, Die Stadt Prostejov ist bei 49.47188 und 17.11184, Die Stadt Kielce ist bei 50.87033 und 20.62752, Die Stadt Kobryn ist bei 52.21173 und 24.3563, Die Stadt Minsk ist bei 53.9 und 27.56667, Die Stadt Vitebsk ist bei 55.1904 und 30.2049, Die Stadt Rzhev ist bei 56.26289 und 34.3289, Die Stadt Moscow ist bei 55.75222 und 37.61556]
Es werden 15 Stationen (inkl. Start & Ende) angefahren und 14 Zwischenstopps eingelegt.
Gewichteter Pfad: [Die Stadt Malaga ist bei 36.72016 und -4.42034, Die Stadt Campina ist bei 38.21896 und -2.98069, Die Stadt Albacete ist bei 38.99424 und -1.

---
Sie haben sicherlich bemerkt, dass für den ungewichteten Fall die Pfadlänge nur der Anzahl an gefahrenen Etappen entspricht.  
Nutzen Sie die hier definierte Hilfsfunktion
```Python
def get_length_from_path(path: list[City]) -> float:
```
Um die tatsächliche Distanz der Route zu bestimmen führen Sie die Zelle dazu davor aus, um die Funktion zu definieren und oben verwenden zu können.

In [130]:
def get_length_from_path(path: list[City]) -> float:
    length = 0.0
    for start, end in zip(path, path[1:]):
        length += graph[start][end]['weight']
    return length

##bis hier hin gelöst

---
### Bestimmen sie die Pfade für die zufällig ausgewählten Zielstädte

In Ihrere Logistikfirma ist jedoch natürlich nicht im vorhinein bekannt, welche Städte angefahren werden müssen.  
Wir bestimmen also 5 zufällige Städte aus unserem Datensatz und bestimmen für diese die kürzeste Route vom Startpunkt Innsbruck.

In [44]:
import sys, random

# Speichern des Seed des Random Generators um wieder die selben Zufallszahlen zu erhalten
seed = random.randrange(sys.maxsize)
random.seed(seed)
random_goals = [random.choice(list(graph.nodes)) for _ in range(5)]

print(random_goals) #Die Musterausgabe unten wird natürlich höchstwahrscheinlich nicht mit Ihrer Ausgabe übereinstimmen

[Die Stadt Suceava ist bei 47.63333 und 26.25, Die Stadt Ozerki ist bei 60.03947 und 30.31128, Die Stadt Horishni Plavni ist bei 49.00835 und 33.62926, Die Stadt Bergedorf ist bei 53.48462 und 10.22904, Die Stadt Langenfeld ist bei 51.10821 und 6.94831]


Iterieren Sie durch alle Zielstädte (`random_goals`) und bestimmen Sie den kürzesten Weg.  
Behandeln Sie auch den Ausnahmefall, dass keine Route gefunden werden kann. Die Dokumentation von `networkx` gibt Ihnen dabei an, welche `Exception` in diesem Fall geworfen wird.

In [43]:
innsbruck = City("Innsbruck", 47.26266, 11.39454)

my_paths = []
for current_goal in random_goals:
    # Lösung hier einfügen
    # Achten Sie auf das Exception-Handling, falls keine mögliche Route gefunden wird!
    
    path_weighted =
    path_weighted_length =

    # Ende der Lösung
    print(F"Pfad von {innsbruck} nach {current_goal} gefunden --> {path_weighted_length} km lang in {len(path_weighted)} Stationen")
    my_paths.append(path_weighted)


SyntaxError: invalid syntax (3165487845.py, line 8)

---
### Wir wollen nun unsere Routen auf einer Karte visualisieren
Dazu soll das Modul `plotly` verwendet werden. Dieses muss, ähnlich wie `networkx` zuvor, erst installiert werden.

Es wird eine eigenen Colormap erstellt, welche jede Route in einer anderen Farbe darstellt.

In [42]:
import matplotlib.pyplot as plt
import matplotlib.colors

def my_color_map(n) -> str:
    color = plt.cm.tab20(n % 20)
    return matplotlib.colors.to_hex(color)



Hier werden für Sie bereits Funktionen definiert, mit denen eine Liste an Städten auf einer Karte visualisiert werden kann.  
Ebenseo kann eine Liste an Pfaden (entspricht einer Liste an Listen an Städten) auf einer Karte visualisiert werden.

Beachten Sie, dass es faktisch nicht möglich ist mit diesen Funktionen alle Kanten des Graphen zu visualisieren, auch wenn dies natürlich sehr spannend wäre. Der dadurch entstehende Output ist schlicht zu groß um stabil in einem Juptyer Notebook dargestellt werden zu können.

ModuleNotFoundError: No module named 'plotly'

### Erstellen der `plotly`-Figure
Hier wird die `plotly`-Figure erstellt, welche die Karte mit den Routen darstellt.

In [44]:
fig = go.Figure()

# Lösung
# Fügen Sie die richtigen Funktionsargumente ein

plot_cities(fig, graph.nodes)
plot_paths(fig, my_paths)

# Ende der Lösung

fig.update_layout(
    height = 600,
    margin = {"r":0,"t":0,"l":0,"b":0},
    showlegend = False,
    geo = dict(
        center=dict(
            lat=51.0057,
            lon=13.7274
        ),
        lataxis_range=[30, 75],
        lonaxis_range=[-20, 70],
        showcountries=True,
        projection_scale=1.5
    )
)
fig.show()

NameError: name 'go' is not defined

---
### Wiederholung für andere Reichweiten
Wiederholen Sie die Schritte für andere, selbstgewählte Reichweiten.  
Um die gleichen zufälligen Ziele zu verwenden, können Sie die Zelle mit der Zufallsauswahl erneut ausführen. Um besser nachvollziehen zu können was gerade passiert sehen Sie unten noch die Ausgabe einer möglichen Implementierung.

Tipp: Speichern Sie alle benötigten Outputs in einen DataFrame um die anschließende Visualisierung zu vereinfachen. Sie können aber auch eine andere Datenstruktur wählen.

In [ ]:
travel_ranges = [150, 200, 300, 400] #die hier angegebenen Reichweiten sind nur ein Vorschlag, Sie können auch andere Werte verwenden
output_data = []

for travel_range in travel_ranges:
    graph.remove_edges_from(list(graph.edges())) # alle Kanten entfernen und in der Lösung wieder für die aktuelle travel_range hinzufügen

    # Lösung hier einfügen

    # Ende der Lösung

output_df = pd.DataFrame(output_data)

# Der Output der unten ausgegeben wird ist wieder nur ein Beispiel, Sie können auch andere Werte erhalten bzw. müssen nichts ausgeben

---
### DataFrame mit Output Daten
So könnte Ihr DataFrame beispielsweise aussehen. Sie können Ihre Daten für die Visualisierung auch anders strukturieren, dies it nur eine mögliche Variante.

In [ ]:
display(output_df)

---
### Visualisierung der Ergebnisse für die unterschiedlichen Routen
Wir wollen uns mit Hilfe von Boxplots ansehen, wie sich die Anzahl der Stopps und die zurückgelegte Distanz für die unterschiedlichen Reichweiten verhält.  
Sie können dabei z.B. direkt die `boxplot(...)`-Methode Ihres DataFrames verwenden.

In [ ]:
# Lösung hier einfügen
# Die Boxplots müssen nicht identisch sein, sollen aber die gleiche Art an Informationen beinhalten

### Visualisierung der Ergebnisse für unterschiedliche Reichweiten
Es wird jeweils der Mittelwert an Stopps bzw. der Mittelwert der zurückgelegten Distanz für die 5 zufälligen Ziele visualisert.

In [ ]:
# Lösung hier einfügen
# Die Plots müssen nicht identisch sein, sollen aber die gleiche Art an Informationen beinhalten
